# TP3: Limpiando y analizando datos complejos

La limpieza de datos es una parte clave en el análisis de datos, pero puede resultar profundamente frustrante. ¿Por qué algunos de los campos de texto están rotos? ¿Qué se debe hacer acerca de los valores nulos o faltantes? ¿Por qué es tan complicado manejar las fechas? ¿Cómo se puede limpiar rápidamente los datos incoherentes o inconsistentes? 

En este TP vamos a trabajar en 3 ejercicios prácticos con datos reales y desordenados que tienen aspectos frecuentes en bases de datos.

Primero trabajaremos con los datos faltantes, luego con las fechas y por último con las incoherencias o inconsistencias. 

Como van a ver las bases de datos son un poco más pesadas que las anteriores con las que veniamos trabajando. De hecho, las bases de datos son muy pesada como para que esten en github. Por lo tanto pueden descargarlas utilizando los siguientes links, recuerden subirlas luego a la carpeta TP3 dentro de IMD en su google drive:

[Faltantes.csv](https://drive.google.com/file/d/1JpoRDVxIt7BgiYn-ZMv2CCLiVQKBX2Ol/view?usp=sharing)

[fechas.csv](https://drive.google.com/file/d/1CuJe82S8Xom7QibCug-v1d2D081OMmTb/view?usp=sharing)

[incosistencias.csv](https://drive.google.com/file/d/1qX1l1BdHqtTzmreEnFlHkMJ7X2W6ykLL/view?usp=sharing)

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

nfl_data = pd.read_csv("/content/drive/MyDrive/IMD/TP3/Fantantes.csv")

np.random.seed(0) 

Lo primero que hay que hacer cuando se obtiene un nuevo conjunto de datos es verlo. Esto te permite ver si se ha leído correctamente. En este caso, veamos si hay valores faltantes, los cuales se representarán con NaN o None.





In [ ]:
nfl_data.head()

La primera base de datos corresponde a estadísticas deportivas, sus campos estas relacionados a cada partido. Parece que hay datos faltantes, ¿no?

## Parte 1: Valores faltantes¶



In [ ]:
missing_values_count = nfl_data.isnull().sum()

missing_values_count[0:10]

Vemos cuantos valroes faltantes hay por columna. Sería útil ver qué porcentaje de los valores de nuestro conjunto de datos faltan para tener una mejor idea de la magnitud de este problema:





In [ ]:
# how many total missing values do we have?
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)


Casi una cuarta parte de las celdas de este conjunto de datos están vacías! En el próximo paso, vamos a ver de más de cerca a algunas de las columnas con valores faltantes e intentar averiguar qué puede estar pasando.

### Identificar la causa de los datos faltantes
Para tratar con valores faltantes, se deberá utilizar la intuición (o conocimiento experto) para averiguar por qué falta el valor. Una de las preguntas más importantes que se puede hacer para ayudar a resolver esto es la siguiente:

¿Este valor falta porque no fue registrado o porque no existe?

Si un valor falta porque no existe (como la altura del hijo mayor de alguien que no tiene hijos), entonces no tiene sentido intentar adivinar qué podría ser. Estos valores probablemente se deban mantener como NaN. Por otro lado, si un valor falta porque no fue registrado, entonces podes intentar adivinar qué podría haber sido basándote en los otros valores de esa columna y fila. 

Veamos un ejemplo. Mirando la cantidad de valores faltantes en el dataframe "nfl_data", notamos que la columna "TimesSec" tiene muchos valores faltantes:

In [ ]:
# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Al mirar la documentación, puede verse que esta columna tiene información sobre el número de segundos restantes en el juego cuando se realizó la jugada. Esto significa que es probable que estos valores falten porque no fueron registrados, en lugar de no existir. Por lo tanto, tendría sentido intentar adivinar qué deberían ser en lugar de dejarlos como NA.

Por otro lado, hay otros campos, como "PenalizedTeam", que también tienen muchos valores faltantes. En este caso, sin embargo, el campo falta porque si no hubo una penalización, no tiene sentido decir qué equipo fue penalizado. Para esta columna, tendría más sentido dejarla vacía o agregar un tercer valor como "ninguno" y usarlo para reemplazar los NA.


### Quitando los valores faltantes
Si estás seguro de que querés eliminar las filas con valores faltantes, pandas tiene una función útil, "dropna()" para ayudarte a hacer esto.

In [ ]:
# remove all the rows that contain a missing value
nfl_data.dropna()


El resultado claramente no es el esperado, nos quedamos sin ninguna fila en la base de datos. Esto se debe a que cada fila en nuestro conjunto de datos tenía al menos un valor faltante. Quizás tengamos más suerte si eliminamos todas las columnas que tienen al menos un valor faltante en su lugar.

In [ ]:
# remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()


In [ ]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])


Ahora no hay valores faltantes, pero con el costo de haber eliminado un montón de datos! O sea pasamos de tener 102 columnas a solamente 41.



### Otra estrategia, reemplazando los valores
También se pueden reemplazar los valores faltantes por algo que no nos afecte, o que lo consideremos en el resultado final.

In [ ]:
# get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data


Podemos usar la función fillna() de Pandas para llenar los valores faltantes en un dataframe. Una opción que tenemos es especificar con qué queremos reemplazar los valores NaN. Acá estoy diciendo que quiero reemplazar todos los valores NaN con 0.





In [ ]:
subset_nfl_data.fillna(0)


También podría una opción reemplazar los valores faltantes con el valor que viene directamente después en la misma columna. (Esto tiene mucho sentido para conjuntos de datos donde las observaciones tienen algún tipo de orden lógico).





In [ ]:
subset_nfl_data.fillna(method='bfill', axis=0).fillna(0)

## Parte 2: trabajando con fechas
Trabajar con fechas puede resultar complejo y tedioso. Lo primero que se tiene que hacer es cargar las bibliotecas y el conjunto de datos que se van a utilizar. Trabajaremos con un conjunto de datos que contiene información sobre terremotos que ocurrieron entre 2007 y 2016. 


In [ ]:
import seaborn as sns
import datetime

# read in our data
landslides = pd.read_csv("/content/drive/MyDrive/IMD/TP3/fechas.csv")

# set seed for reproducibility
np.random.seed(0)

### Como siempre, primero vemos los datos

In [ ]:
landslides.head()


Trabajaremos con la columna "date" del dataframe de deslizamientos de tierra. Asegurémonos de que realmente parece contener fechas.





In [ ]:
# print the first few rows of the date column
print(landslides['date'].head())

Es posible que tengas que revisar la documentación de numpy para hacer coincidir el código de letras con el dtype del objeto. "O" es el código para "object", por lo que podemos ver que estos dos métodos nos dan la misma información.


### Convertir el tipo date a datetime (manejable)
Ahora que sabemos que nuestra columna de fecha no está siendo reconocida como una fecha, es hora de convertirla para que se reconozca como tal. Esto se llama "analizar fechas" porque estamos tomando un texto e identificando sus partes componentes. Tengan en cuenta que se requiere explícitar todo, y que hay múltiples formas de escribir las fechas, pensemos en el ej. en cuestión y las fechas que estamos acostumbrados a anotar.

Podemos determinar cuál es el formato de nuestras fechas con una guía llamada "strftime". La idea básica es que hay señalar qué partes de la fecha son y qué puntuación hay entre ellas. Hay muchas partes posibles de una fecha, pero las más comunes son %d para el día, %m para el mes, %y para un año de dos dígitos y %Y para un año de cuatro dígitos.

Algunos ejemplos:

1/17/07 tiene el formato "%m/%d/%y"
17-1-2007 tiene el formato "%d-%m-%Y"
Mirando de nuevo la cabecera de la columna "date" en el conjunto de datos de terremotos, podemos ver que está en el formato "mes/día/año" de dos dígitos, por lo que podemos utilizar la misma sintaxis que el primer ejemplo para analizar nuestras fechas:





In [ ]:
# create a new column, date_parsed, with the parsed dates
landslides['date_parsed'] = pd.to_datetime(landslides['date'], format="%m/%d/%y")

Ahora, al verificar las primeras filas de la nueva columna, se puede ver que el tipo de datos es datetime64. También las fechas han sido ligeramente reorganizadas para que se ajusten al orden predeterminado de los objetos datetime (año-mes-día).

In [ ]:
# print the first few rows
landslides['date_parsed'].head()

¿Qué pasa si me encuentro con un base con múltiples formatos de fecha? Mientras especificamos el formato de fecha aquí, a veces te encontrarás con un error cuando hay múltiples formatos de fecha en una sola columna. Si eso sucede, se puede hacer que pandas intente inferir cuál debería ser el formato de fecha correcto. Podrías hacerlo de la siguiente manera:
landslides['date_parsed'] = pd.to_datetime(landslides['Date'], infer_datetime_format=True)

¿Por qué no siempre usamos infer_datetime_format = True? (que es la opción que permite adivinar). Hay dos grandes razones por las que no siempre es recomendable hacer que pandas adivine el formato de fecha. La primera es que pandas no siempre podrá determinar el formato de fecha correcto, especialmente si alguien es muy creativo con la carga de datos. La segunda es que es mucho más lento que especificar el formato exacto de las fechas.

### Seleccionar el día del mes
Ahora que tenemos una columna de fechas analizadas, podemos extraer información como el día del mes en que ocurrió un terremoto.

In [ ]:
# get the day of the month from the date_parsed column
day_of_month_landslides = landslides['date_parsed'].dt.day
day_of_month_landslides.head()

Si intentamos obtener la misma información de la columna original "date", obtendríamos un error: "AttributeError: Can only use .dt accessor with datetimelike values". Esto se debe a que dt.day no sabe cómo manejar una columna con el tipo de datos "object". Aunque nuestro dataframe tiene fechas, debemos analizarlas antes de poder interactuar con ellas de manera útil.

### Gráficar las fechas para chequear que funcionó
Uno de los mayores peligros al analizar fechas es mezclar los días y los meses. La función to_datetime() tiene mensajes de error muy útiles, pero no está de más verificar que los días del mes que hemos extraído tengan sentido.

Para ello, vamos a graficar un histograma de los días del mes. Esperamos que tenga valores entre 1 y 31 y, dado que no hay razón para suponer que los terremotos son más comunes en algunos días del mes que en otros, una distribución relativamente uniforme (con una caída en el día 31 porque no todos los meses tienen 31 días). Veamos si ese es el caso:





In [ ]:
# remove na's
day_of_month_landslides = day_of_month_landslides.dropna()

# plot the day of the month
sns.histplot(day_of_month_landslides, kde=False, bins=31)

## Parte3: Inconsistencias
Acá vamos a chequear datos que pueden estar mal cargados o no tienen sentido.

In [ ]:
!pip3 install fuzzywuzzy
import fuzzywuzzy
from fuzzywuzzy import process
import charset_normalizer

professors = pd.read_csv("/content/drive/MyDrive/IMD/TP3/inconsistencias.csv")

np.random.seed(0)

### Vamos a hacer un pre-procesamiento

In [ ]:
professors.head()

Como pueden ver, la base de datos contiene información de docentes. Digamos que estamos interesados en limpiar la columna "País" para asegurarnos de que no haya inconsistencias en la entrada de datos. Podríamos revisar cada fila manualmente y corregir a mano las inconsistencias que encontremos. El problema acá es que el análisis manual puede demorar mucho tiempo, cuando no volverse practicamente imposible si tenemos bases de datos de millones de registros. Existe una forma automática más sencilla.**texto en negrita**

In [ ]:
countries = professors['Country'].unique()

countries.sort()
countries

Como se ve hay nombres de países, que son tomados como Object. Inspeccionaremos uno a uno. Primero se va a poner todo en minúsculas (puede cambiarse al final si lo desea) y eliminar cualquier espacio en blanco al principio y al final de las celdas. Las inconsistencias en las mayúsculas y los espacios en blanco finales son muy comunes en los datos de texto y se pueden solucionar alrededor del 80% de las inconsistencias en los datos de texto haciendo esto.

In [ ]:
professors['Country'] = professors['Country'].str.lower()
professors['Country'] = professors['Country'].str.strip()

### Usamos la coincidencia difusa para corregir la entrada inconsistente de datos.
De acuerdo, echemos otro vistazo a la columna 'Country' y veamos si necesitamos hacer más limpieza de datos.

In [ ]:
countries = professors['Country'].unique()
countries.sort()
countries

Parece que hay otra inconsistencia: 'southkorea' y 'south korea' deberían ser lo mismo.

Vamos a usar el paquete fuzzywuzzy para ayudarnos a identificar cuáles cadenas de texto son más similares entre sí.

Fuzzy matching: El proceso de encontrar automáticamente cadenas de texto que son muy similares a la cadena de texto objetivo. En general, se considera que una cadena de texto está "más cerca" de otra cuanto menos cambios necesitas hacer para transformar una cadena de texto en otra. Entonces "apple" y "snapple" están a dos cambios de distancia (agregar "s" y "n") mientras que "in" y "on" están a un cambio de distancia (reemplazar "i" por "o"). No siempre podrás confiar en el fuzzy matching al 100%, pero generalmente te ahorrará al menos un poco de tiempo.

Fuzzywuzzy devuelve una relación entre dos cadenas de texto. Cuanto más cercana sea la relación a 100, menor será la distancia de edición entre las dos cadenas. Aquí, vamos a obtener las diez cadenas de texto de nuestra lista de ciudades que tienen la distancia más cercana a "south korea".

In [ ]:
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

Podemos ver que dos de los elementos en la lista de ciudades son muy similares a "south korea": "south korea" y "southkorea". Reemplazaremos todas las filas en nuestra columna "Country" que tengan una relación > 47 con "south korea".

Para hacer esto, voy a escribir una función. (Es una buena idea escribir una función de propósito general que se pueda reutilizar si se piensa que se tendrá que realizar una tarea específica más de una o dos veces. Esto evita tener que copiar y pegar el código con demasiada frecuencia, lo que ahorra tiempo y puede ayudar a prevenir errores.)

NOTA: es la primera vez en el práctico que definimos una función, asi que vamos a discutirla!

In [ ]:
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    strings = df[column].unique()    
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    rows_with_matches = df[column].isin(close_matches)
    df.loc[rows_with_matches, column] = string_to_match    
    print("Hecho!")

Ahora que tenemos una función, ¡podemos ponerla a prueba!


In [ ]:
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")

Y ahora también chequeamos:

In [ ]:
countries = professors['Country'].unique()
countries.sort()
countries

¡Excelente! Ahora solo tenemos "south korea" en nuestro dataframe y no tuvimos que cambiar nada a mano.